In [ ]:
# Import necessary initial libraries
import os
import sys
import logging
import re
from datetime import timedelta
import plotly.express as px

In [ ]:
# Sets the root directory of the project as the working directory
os.chdir('..')

In [ ]:
# Check the current working directory
os.getcwd()

In [5]:
# Import the custom module
from src import merge_geojsonGrids

In [6]:
# Reload the module to ensure any changes are reflected
import importlib
from importlib import reload

importlib.reload(merge_geojsonGrids)

<module 'src.merge_geojsonGrids' from '/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/src/merge_geojsonGrids.py'>

### Consolidação dos arquivos GeoJSON das Grades Geoespaciais, resultantes de cada step, em um único arquivo

In [8]:
# Lista dos arquivos GeoJSON para merge (ordem importa - primeiro arquivo tem prioridade)
# Usando pathlib para tornar o código mais legível e robusto
from pathlib import Path

testes_dir = Path(os.getcwd()) / 'data/output_code1/4cells_tests'
output_code1_files = sorted([str(f) for f in testes_dir.glob('*.geojson')])
output_code1_files


['/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/data/output_code1/4cells_tests/step1_consolidado.geojson',
 '/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/data/output_code1/4cells_tests/step2_consolidado.geojson',
 '/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/data/output_code1/4cells_tests/step4_consolidado.geojson',
 '/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/data/output_code1/4cells_tests/step5_consolidado.geojson',
 '/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/data/output_code1/4cells_tests/step6_consolidado.geojson',
 '/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/data/output_code1/4cells_tests/steps_consolidado.geojson']

In [ ]:
# Salva os arquivos GeoJSON consolidados em um arquivo GeoPackage
output_file_name = "steps_consolidado.gpkg"
output_path = os.path.join(testes_dir, output_file_name)

merge_geojsonGrids.merge_geojson2gpkg(output_code1_files, str(output_path))

In [ ]:
# Salva os arquivos GeoJSON consolidados em um único arquivo GeoJSON
output_file_name = "steps_consolidado.geojson"
output_path = os.path.join(testes_dir, output_file_name)

merge_geojsonGrids.merge_geojson2geojson(output_code1_files, str(output_path))

### 3. Métricas de Tempo

In [ ]:
# Leitura do Arquivo Excel
file_name = [
    "results/1_output_grid/time_metrics/time_log_OHSOME_requests.xlsx"
]

In [ ]:
# Leitura do Arquivo Excel
df_time = pd.read_excel(file_name[0], sheet_name='step1')

In [ ]:
# Exibe as primeiras linhas para validar
display(df_time.head(10))

In [ ]:
# Exibe as últimas linhas para validar
display(df_time.tail(10))

In [ ]:
# Função para extrair tempos dos lotes
def extrair_tempos(df):
    resultados = []

    for line in df.iloc[:, 0].dropna():
        match = re.search(r'Tempo do lote (\d+): (\d+\.?\d*) min (\d+\.?\d*) seg', line)
        if match:
            lote = int(match.group(1))
            minutos = float(match.group(2))
            segundos = float(match.group(3))
            total_segundos = int(minutos * 60 + segundos)
            resultados.append({
                'Lote': lote,
                'Minutos': minutos,
                'Segundos': segundos,
                'Total (s)': total_segundos,
                'Total (min)': total_segundos / 60,
                'Total (h)': total_segundos / 3600,
                'Total (dias)': total_segundos / (3600 * 24),
                'Duração (timedelta)': timedelta(seconds=total_segundos)
            })

    return pd.DataFrame(resultados)

In [ ]:
# Aplicar a função para extrair os tempos
df_tempos = extrair_tempos(df_time)
df_tempos

In [ ]:
# Cálculo do tempo total consolidado de todos os lotes
total_horas = df_tempos['Total (h)'].sum()
total_dias = df_tempos['Total (dias)'].sum()
print(f"Tempo Total (todos os lotes): {total_horas :.2f} horas")
print(f"Tempo Total (todos os lotes): {total_dias :.2f} dias")

In [ ]:
# Gráfico de barras para visualizar os tempos de processamento
fig = px.bar(
    df_tempos,
    x='Lote',
    y='Total (min)',
    hover_data={
        'Minutos': ':.1f',
        'Segundos': ':.1f',
        'Total (h)': ':.3f'
    },
    labels={'Total (min)': 'Duração (minutos)'},
    title='Tempo de Processamento por Lote (20 células) - Step 1 (name_ratio)',
)

# Cálculo do tempo total consolidado de todos os lotes
total_horas = df_tempos['Total (h)'].sum()
total_dias = df_tempos['Total (dias)'].sum()

# Adiciona anotação com a duração total
fig.add_annotation(
    xref="paper", 
    yref="paper",
    x=0.5, 
    y=-0.35,  # Ajuste a posição vertical aqui
    text=f"Duração Total: <br>({total_horas:.2f} horas / {total_dias:.2f} dias)",
    showarrow=False,
    font=dict(size=12),
    align="center"
)

# Ajustes estéticos
fig.update_traces(marker_color='royalblue')
fig.update_layout(
    title_x=0.5,
    xaxis_title='Lote',
    yaxis_title='Duração (minutos)',
    hoverlabel=dict(bgcolor="white", font_size=13),
    margin=dict(b=100)  # Aumenta a margem inferior para caber a anotação
)

# Salva o gráfico em formato HTML
fig.write_html("step1_tempo_processamento.html")

fig.show()